# 物理对象: 闸门 (Gate)

`闸门 (Gate)` 类模拟水系统中的一个可控闸门。其出流量是根据上游和下游的水位以及闸门开启面积，使用孔口方程计算的。

该模型的一个关键特性是 `max_rate_of_change` 参数，它模拟了闸门开启或关闭速度的物理限制。闸门的实际开启度将以这个有限的速率向其目标值 (`control_signal`) 变化。

### 状态变量
- `opening` (float): 闸门的当前开启度 (例如，单位为米或百分比)。
- `outflow` (float): 计算得出的通过闸门的出流量 (m³/s)。

### 参数
- `discharge_coefficient` (float): 孔口方程中的一个无量纲系数C。
- `width` (float): 闸门开口的宽度 (m)。
- `max_opening` (float): 可能的最大开启值。
- `max_rate_of_change` (float): 闸门开启度每秒可以变化的最大速度。

## 仿真示例

以下示例模拟了一个被给予一系列目标开启度的闸门。我们可以观察到实际开启度如何因 `max_rate_of_change` 的限制而逐渐趋向目标值，以及这如何影响出流量。为隔离闸门的行为，上游和下游水位保持不变。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.gate import Gate
from swp.core.interfaces import State, Parameters

# 1. 定义初始状态和参数
initial_state = State(opening=0.1)
parameters = Parameters(
    discharge_coefficient=0.6,
    width=10.0,
    max_opening=1.0,
    max_rate_of_change=0.05 # 每秒只能改变5%
)

# 2. 创建一个Gate实例
gate = Gate(name="control_gate", initial_state=initial_state, parameters=parameters)

# 3. 仿真设置
dt = 1.0
simulation_duration = 100
num_steps = int(simulation_duration / dt)
history = []
target_openings = []

# 4. 运行仿真循环
for t in range(num_steps):
    # 为目标开启度创建一个变化的控制信号
    if t < 20:
        target = 0.8
    elif t < 60:
        target = 0.2
    else:
        target = 1.0
    target_openings.append(target)
    
    action = {
        'control_signal': target,
        'upstream_head': 10.0, # 恒定的上游水位
        'downstream_head': 8.0  # 恒定的下游水位
    }
    
    current_state = gate.step(action=action, dt=dt)
    history.append(current_state.copy())

print("闸门仿真完成。")

## 结果与可视化

图表显示了 `target_opening` (控制信号) 与闸门的实际 `opening` (开启度) 的对比。由于 `max_rate_of_change` 的限制，开启度逐渐变化的 ramping 行为清晰可见。`outflow` (出流量) 图显示了流量如何因闸门的移动而变化。

In [ ]:
# 从历史记录中提取数据
time = [i * dt for i in range(num_steps)]
openings = [h['opening'] for h in history]
outflows = [h['outflow'] for h in history]

# 创建一个DataFrame
df = pd.DataFrame({
    'Time': time,
    'Target Opening': target_openings,
    'Actual Opening': openings,
    'Outflow': outflows
})

print(df.head())

# 绘制结果
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# 闸门开启度
ax1.plot(df['Time'], df['Target Opening'], label='目标开启度 (控制信号)', color='red', linestyle='--')
ax1.plot(df['Time'], df['Actual Opening'], label='实际开启度', color='blue')
ax1.set_ylabel('开启度 (%)')
ax1.set_title('闸门动态过程')
ax1.grid(True)
ax1.legend()

# 出流量
ax2.plot(df['Time'], df['Outflow'], label='出流量', color='green')
ax2.set_xlabel('时间 (s)')
ax2.set_ylabel('出流量 (m³/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()